In [ ]:
# Single run
from floor_generator import *

hyperparams = {
    'east_joists' : MemberSpec('c24_60x120', quantity=1, padding=100),
    'west_joists' : MemberSpec('c24_60x120', quantity=1, padding=200),
    'tail_joists' : MemberSpec('c24_60x120', quantity=1, padding=250),
    'trimmers' : MemberSpec('c24_60x120', quantity=2),
    'header' : MemberSpec('c24_60x120', quantity=1),
    'planks' : MemberSpec('c18_200x25'),
    }

frame, nodes, members = create_model(hyperparams, walls=True)
part_evaluations = evaluate_stresses(frame, members)
total_cost, cuts = calculate_purchase_quantity(frame, members)
member_evaluations = group_stresses_by_member(part_evaluations)
nodes

In [ ]:
viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_quantity, max_east_quantity, max_tail_quantity = 3, 3, 4

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_padding = west_space - (beam_max_base * max_west_quantity)
max_east_padding = east_space - (beam_max_base * max_east_quantity)
max_tail_padding = (tail_space - (beam_max_base * max_tail_quantity))/2
max_tail_padding

In [ ]:
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
import multiprocessing as mp
from floor_generator import *

def objective_function(east_material, east_quantity, east_padding,
                      west_material, west_quantity, west_padding,
                      tail_material, tail_quantity, tail_padding,
                      trimmer_material, header_material, plank_material):
    
    param_dict = {
        'east_material': east_material,
        'east_quantity': east_quantity,
        'east_padding': east_padding,
        'west_material': west_material,
        'west_quantity': west_quantity,
        'west_padding': west_padding,
        'tail_material': tail_material,
        'tail_quantity': tail_quantity,
        'tail_padding': tail_padding,
        'trimmer_material': trimmer_material,
        'header_material': header_material,
        'plank_material': plank_material,
    }
    
    try:
        hyperparams = {
            'east_joists': MemberSpec(east_material, quantity=east_quantity, padding=east_padding),
            'west_joists': MemberSpec(west_material, quantity=west_quantity, padding=west_padding),
            'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
            'trimmers': MemberSpec(trimmer_material, quantity=2),
            'header': MemberSpec(header_material, quantity=1),
            'planks': MemberSpec(plank_material),
        }

        frame, _, members = create_model(hyperparams, walls=True)
        part_evaluations = evaluate_stresses(frame, members)
        total_cost, cuts = calculate_purchase_quantity(frame, members)
        member_evaluations = group_stresses_by_member(part_evaluations)

        # Optimize to target ratio
        target = 1
        above_target_multiplier = 2.0
        below_target_multiplier = 1.0

        diff = member_evaluations - target
        multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
        member_evaluations_penalty = multiplier * (diff * diff)

        variety_factor = math.log(len(cuts), 99) + 1

        max_ratio = member_evaluations_penalty.max().max()
        mean_ratio = member_evaluations_penalty.mean().mean()
        score = total_cost * mean_ratio * max_ratio * variety_factor

    except Exception as e:
        warnings.warn(f"An exception occurred with params {param_dict}: {e}")
        score = 1e12
        total_cost = float('inf')
        member_evaluations = pd.DataFrame()
        cuts = {}
        max_ratio = float('inf')
        mean_ratio = float('inf')

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# Units are mm, N, and MPa (N/mm²)
# INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_quantity, max_east_quantity, max_tail_quantity = 3, 3, 4

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_padding = west_space - (beam_max_base * max_west_quantity)
max_east_padding = east_space - (beam_max_base * max_east_quantity)
max_tail_padding = tail_space - (beam_max_base * max_tail_quantity)

space_config = {
    'east_material': {'type': 'categorical', 'choices': beam_ids},
    'east_quantity': {'type': 'integer', 'lower': 1, 'upper': max_east_quantity},
    'east_padding': {'type': 'integer', 'lower': 0, 'upper': max_east_padding},
    
    'west_material': {'type': 'categorical', 'choices': beam_ids},
    'west_quantity': {'type': 'integer', 'lower': 1, 'upper': max_west_quantity},
    'west_padding': {'type': 'integer', 'lower': 0, 'upper': max_west_padding},
    
    'tail_material': {'type': 'categorical', 'choices': beam_ids},
    'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
    'tail_padding': {'type': 'integer', 'lower': 0, 'upper': max_tail_padding},
    
    'trimmer_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
    'header_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
    'plank_material': {'type': 'categorical', 'choices': floor_ids},
}

n_calls = 2000
_GLOBAL_EVALUATIONS = []

result = optimize_with_nevergrad(
    space_config, 
    n_calls=n_calls,
    n_workers=6,
    algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
)

results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df = results_df.sort_values(by='score')
results_df

In [ ]:
import nevergrad as ng
import pandas as pd
import numpy as np
import warnings
import math
from tqdm import tqdm
import multiprocessing as mp
from floor_generator import *

def objective_function(east_material, east_padding,
                      west_material, west_padding,
                      tail_material, tail_quantity, tail_padding,
                      trimmer_material, header_material):
    
    param_dict = {
        'east_material': east_material,
        'east_padding': east_padding,
        'west_material': west_material,
        'west_padding': west_padding,
        'tail_material': tail_material,
        'tail_quantity': tail_quantity,
        'tail_padding': tail_padding,
        'trimmer_material': trimmer_material,
        'header_material': header_material,
    }
    
    try:
        hyperparams = {
            'east_joists': MemberSpec(east_material, quantity=1, padding=east_padding),
            'west_joists': MemberSpec(west_material, quantity=1, padding=west_padding),
            'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
            'trimmers': MemberSpec(trimmer_material, quantity=2),
            'header': MemberSpec(header_material, quantity=1),
            'planks': MemberSpec('c18_200x25'),
        }

        frame, _, members = create_model(hyperparams, walls=True)
        part_evaluations = evaluate_stresses(frame, members)
        total_cost, cuts = calculate_purchase_quantity(frame, members, skip_planks=True)
        member_evaluations = group_stresses_by_member(part_evaluations)

        # Optimize to target ratio
        target = 1
        above_target_multiplier = 2.0
        below_target_multiplier = 1.0

        diff = member_evaluations - target
        multiplier = np.where(diff > 0, above_target_multiplier, below_target_multiplier)
        member_evaluations_penalty = multiplier * (diff * diff)

        variety_factor = math.log(len(cuts), 99) + 1

        max_ratio = member_evaluations_penalty.max().max()
        mean_ratio = member_evaluations_penalty.mean().mean()
        score = total_cost * mean_ratio * max_ratio * variety_factor

    except Exception as e:
        warnings.warn(f"An exception occurred with params {param_dict}: {e}")
        score = 1e12
        total_cost = float('inf')
        member_evaluations = pd.DataFrame()
        cuts = {}
        max_ratio = float('inf')
        mean_ratio = float('inf')

    run_result = {
        **param_dict,
        'total_cost': total_cost,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'cuts': cuts,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS.append(run_result)

    return score


def optimize_with_nevergrad(space_config, n_calls=2048, n_workers=1, algorithm='NGOpt'):
    param_dict = {}
    for param_name, param_spec in space_config.items():
        if param_spec['type'] == 'categorical':
            param_dict[param_name] = ng.p.Choice(param_spec['choices'])
        elif param_spec['type'] == 'integer':
            param_dict[param_name] = ng.p.Scalar(lower=param_spec['lower'], upper=param_spec['upper']).set_integer_casting()

    parametrization = ng.p.Instrumentation(**param_dict)
    
    # Select optimizer
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=n_workers)
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function(**x.kwargs)
        optimizer.tell(x, value)
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# Units are mm, N, and MPa (N/mm²)
# INPUT_PARAMS, MATERIAL_STRENGTHS, MATERIAL_CATALOG, CONNECTORS, EUROCODE_FACTORS = prep_data()

viable_beams = MATERIAL_CATALOG[MATERIAL_CATALOG['viable_connector']]
beam_ids = viable_beams[viable_beams['type'] == 'beam']['id'].unique().tolist()
double_ids = viable_beams[viable_beams['type'] == 'double']['id'].unique().tolist()
floor_ids = MATERIAL_CATALOG[MATERIAL_CATALOG['type'] == 'floor']['id'].unique().tolist()

west_space = INPUT_PARAMS.opening_x_start
east_space = INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)
tail_space = INPUT_PARAMS.opening_length

max_west_padding = math.ceil(west_space / 1.13)
max_east_padding = math.ceil(east_space / 1.17)
max_tail_padding = math.ceil(tail_space / 1.3)

beam_max_base = MATERIAL_CATALOG[MATERIAL_CATALOG['id'].isin(beam_ids)].base.max()
max_west_quantity = math.floor((west_space - max_west_padding) / beam_max_base)
max_east_quantity = math.floor((east_space - max_east_padding) / beam_max_base)
max_tail_quantity = math.floor((tail_space - max_tail_padding) / beam_max_base)

space_config = {
    'east_material': {'type': 'categorical', 'choices': beam_ids},
    'east_padding': {'type': 'integer', 'lower': 0, 'upper': max_east_padding},
    
    'west_material': {'type': 'categorical', 'choices': beam_ids},
    'west_padding': {'type': 'integer', 'lower': 0, 'upper': max_west_padding},
    
    'tail_material': {'type': 'categorical', 'choices': beam_ids},
    'tail_quantity': {'type': 'integer', 'lower': 1, 'upper': max_tail_quantity},
    'tail_padding': {'type': 'integer', 'lower': 0, 'upper': max_tail_padding},
    
    'trimmer_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
    'header_material': {'type': 'categorical', 'choices': beam_ids + double_ids},
}

n_calls = 2000
_GLOBAL_EVALUATIONS = []

result = optimize_with_nevergrad(
    space_config, 
    n_calls=n_calls,
    n_workers=6,
    algorithm='NGOpt'  # Try also: 'CMA', 'NGOpt', 'TwoPointsDE', 'PSO', 'DE'
)

results_df = pd.DataFrame(_GLOBAL_EVALUATIONS)
results_df = results_df.sort_values(by='score')
results_df

In [ ]:
results_df = results_df.drop(columns=['cuts', 'member_evaluations'])
results_df.to_csv('CMA.csv', index=False)

In [ ]:
# Padding optimization
def objective_function_padding(east_padding, west_padding, tail_padding,
                                east_material, east_quantity,
                                west_material, west_quantity,
                                tail_material, tail_quantity,
                                trimmer_material, header_material, plank_material):
    """
    Objective function for padding optimization.
    Minimizes mean_ratio * max_ratio with target of 0.
    """
    
    param_dict = {
        'east_padding': east_padding,
        'west_padding': west_padding,
        'tail_padding': tail_padding,
    }
    
    try:
        hyperparams = {
            'east_joists': MemberSpec(east_material, quantity=east_quantity, padding=east_padding),
            'west_joists': MemberSpec(west_material, quantity=west_quantity, padding=west_padding),
            'tail_joists': MemberSpec(tail_material, quantity=tail_quantity, padding=tail_padding),
            'trimmers': MemberSpec(trimmer_material, quantity=2),
            'header': MemberSpec(header_material, quantity=1),
            'planks': MemberSpec(plank_material),
        }

        frame, _, members = create_model(hyperparams, walls=True)
        part_evaluations = evaluate_stresses(frame, members)
        member_evaluations = group_stresses_by_member(part_evaluations)

        # Target is 0 (perfect utilization)
        target = 0
        diff = member_evaluations - target
        member_evaluations_penalty = diff * diff

        max_ratio = member_evaluations_penalty.max().max()
        mean_ratio = member_evaluations_penalty.mean().mean()
        score = mean_ratio * max_ratio

    except Exception as e:
        warnings.warn(f"An exception occurred with params {param_dict}: {e}")
        score = 1e12
        member_evaluations = pd.DataFrame()
        max_ratio = float('inf')
        mean_ratio = float('inf')

    run_result = {
        **param_dict,
        'max_ratio': max_ratio,
        'mean_ratio': mean_ratio,
        'score': score,
        'member_evaluations': member_evaluations,
    }
    _GLOBAL_EVALUATIONS_PADDING.append(run_result)

    return score


def optimize_padding(best_params, max_east_padding, max_west_padding, max_tail_padding,
                     n_calls=512, algorithm='NGOpt'):
    """
    Second stage optimization: optimize padding given fixed materials and quantities.
    """
    
    space_config = {
        'east_padding': ng.p.Scalar(lower=0, upper=max_east_padding).set_integer_casting(),
        'west_padding': ng.p.Scalar(lower=0, upper=max_west_padding).set_integer_casting(),
        'tail_padding': ng.p.Scalar(lower=0, upper=max_tail_padding).set_integer_casting(),
    }
    
    # Add fixed parameters as constants
    fixed_params = {
        'east_material': best_params['east_material'],
        'east_quantity': best_params['east_quantity'],
        'west_material': best_params['west_material'],
        'west_quantity': best_params['west_quantity'],
        'tail_material': best_params['tail_material'],
        'tail_quantity': best_params['tail_quantity'],
        'trimmer_material': best_params['trimmer_material'],
        'header_material': best_params['header_material'],
        'plank_material': best_params['plank_material'],
    }
    
    parametrization = ng.p.Instrumentation(**space_config, **fixed_params)
    
    optimizer_class = getattr(ng.optimizers, algorithm)
    optimizer = optimizer_class(parametrization=parametrization, budget=n_calls, num_workers=1)
    
    for _ in tqdm(range(n_calls)):
        x = optimizer.ask()
        value = objective_function_padding(**x.kwargs)
        optimizer.tell(x, value)
    
    recommendation = optimizer.provide_recommendation()
    
    return {
        'best_params': recommendation.kwargs,
        'best_score': optimizer.current_bests["minimum"].mean,
        'optimizer': optimizer
    }


# Assume you have the best_params from the first optimization
# best_params = result['best_params']

# Define max padding values (same as first optimization)
max_west_padding = math.ceil(INPUT_PARAMS.opening_x_start / 2)
max_east_padding = math.ceil((INPUT_PARAMS.room_length - (INPUT_PARAMS.opening_x_start + INPUT_PARAMS.opening_length)) / 2)
max_tail_padding = math.ceil(INPUT_PARAMS.opening_length / 3)

_GLOBAL_EVALUATIONS_PADDING = []

# Run second stage optimization
result_padding = optimize_padding(
    best_params=best_params,
    max_east_padding=max_east_padding,
    max_west_padding=max_west_padding,
    max_tail_padding=max_tail_padding,
    n_calls=512,
    algorithm='NGOpt'
)

# View results
results_df_padding = pd.DataFrame(_GLOBAL_EVALUATIONS_PADDING)
results_df_padding.sort_values(by='score')